<a href="https://colab.research.google.com/github/hillwalking/2025-2-Data-Analysis/blob/main/ecomileage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
# @title
import requests
url = 'http://openapi.seoul.go.kr:8088/6f546371447a6e6f3531544a726b62/json/energyUseDataSummaryInfo/1/1000'

response = requests.get(url)

if response.status_code == 200:
    print('API 호출 성공')
    # json data 저장
    data = response.json()
    # data 확인- 1
    print(type(data), ' , data length :  ', len(data))
else:
    print(f'API 호출 실패 : {response.status_code}')

# data 확인- 2
data

# json data 딕셔녀리 형태 확인{'energyUseDataSummaryInfo': {'list_total_count', 'RESULT', 'row' }
# 필요한 row 만 추출 저장함
data = data['energyUseDataSummaryInfo']['row']

# json data를 파일로 저장하기
with open('ecoSeoul.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii = False)

# 데이터프레임으로 변환
df = pd.DataFrame(data)
print('\n---DataFrame head(3)-----\n',df.head(3))


API 호출 성공
<class 'dict'>  , data length :   1

---DataFrame head(3)-----
    YEAR MON  MM_TYPE      CNT        EUS       EUS1       EUS2     ECO2_1  \
0  2025  06       개인  1269454  278745288  275851011  273151435    4244065   
1  2025  06       학교     6745  117808793  114793127  114623246  3100606.5   
2  2025  06  공동주택관리소     3729  498863841  496003195  488758810  6482838.5   

        ECO2_2       GUS  ...        WUS1        WUS2     WCO2_1       WCO2_2  \
0  2076623.656  13084768  ...  15166284.8  15371662.2  -341289.9  -611822.446   
1  3096973.668   2093441  ...   1914804.2   1956560.2    -111344  -333633.134   
2    6482838.5   2853437  ...  20616193.4  20794568.4  -297568.2    -892704.6   

         HUS        HUS1        HUS2    HCO2_1           HCO2_2  \
0  379182.22  368839.049  389847.167  -160.888  -22448.67848156   
1       8.03        8.03        8.03         0                0   
2          0           0           0         0                0   

                REG_DAT

In [28]:
# @title 서울시 에코마일리지 에너지 사용량 개인유형
import requests
import json
import pandas as pd

url = 'http://openapi.seoul.go.kr:8088/6f546371447a6e6f3531544a726b62/json/energyUseDataSummaryInfo/1/1000'



response = requests.get(url)
if response.status_code == 200:
    print('API 호출 성공')
    # json data 저장
    data = response.json()
    # data 확인- 1
    print(type(data), ' , data length :  ', len(data))
else:
    print(f'API 호출 실패 : {response.status_code}')
    # data 확인- 2
data

# 딕셔녀리 형태 확인{'energyUseDataSummaryInfo': {'list_total_count', 'RESULT', 'row' }
data = data['energyUseDataSummaryInfo']['row']

# json data를 파일로 저장하기
with open('ecoSeoul.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii = False)

# 데이터프레임으로 변환
df = pd.DataFrame(data)
print('\n---DataFrame head(3)-----\n',df.head(3))

# 필요한 컬럼만 선택
df = df.loc[:,['YEAR','MON','EUS','GUS','WUS','HUS','MM_TYPE']]

print('\n\n---필요한 컬럼 선택 및 사용자 유형 확인-----\n','data length :  ', len(df))
print(df.head(3))
print('..................\n',df.tail(3))

# 개인 유형 데이터 추출한 후, 'MM_TYPE' 삭제
df_p = df.loc[df['MM_TYPE']=='개인']
df_p = df_p.drop(['MM_TYPE'], axis=1)

# 년 월 데이터 형을 숫자로 변환하기
df_p['YEAR'] = df_p['YEAR'].apply(lambda  x: int(x))
df_p['MON'] = df_p['MON'].apply(lambda  x: int(x))

# 2015년부터 2024년까지 행 추출
df_p = df_p[(df_p['YEAR'] > 2014) & (df_p['YEAR'] < 2025)]

# 인덱스 정리
df_p = df_p.reset_index(drop=True)

# 월 범위에 따라 계절 지정하기
df_p.loc[(df_p['MON'] ==3) | (df_p['MON'] ==4) | (df_p['MON'] ==5), 'SEASON'] = '봄'
df_p.loc[(df_p['MON'] ==6) | (df_p['MON'] ==7) | (df_p['MON'] ==8), 'SEASON'] = '여름'
df_p.loc[(df_p['MON'] ==9) | (df_p['MON'] ==10) | (df_p['MON'] ==11), 'SEASON'] = '가을'
df_p.loc[(df_p['MON'] ==1) | (df_p['MON'] ==2) | (df_p['MON'] ==12), 'SEASON'] = '겨울'

# 컬럼명 한글로 바꾸기
df_p= df_p.rename(columns={'YEAR':'연도', 'MON':'월', 'EUS':'전기', 'GUS':'가스', 'WUS':'수도', 'HUS':'지역난방', 'SEASON':'계절'})

print('\n\n--------- 개인 데이터 연도 및 계절 처리 완료 -----------------\n','data length :  ', len(df_p))
print(df_p.head(7))
print('\n--------------------------\n',df_p.tail(7))




API 호출 성공
<class 'dict'>  , data length :   1

---DataFrame head(3)-----
    YEAR MON  MM_TYPE      CNT        EUS       EUS1       EUS2     ECO2_1  \
0  2025  06       개인  1269454  278745288  275851011  273151435    4244065   
1  2025  06       학교     6745  117808793  114793127  114623246  3100606.5   
2  2025  06  공동주택관리소     3729  498863841  496003195  488758810  6482838.5   

        ECO2_2       GUS  ...        WUS1        WUS2     WCO2_1       WCO2_2  \
0  2076623.656  13084768  ...  15166284.8  15371662.2  -341289.9  -611822.446   
1  3096973.668   2093441  ...   1914804.2   1956560.2    -111344  -333633.134   
2    6482838.5   2853437  ...  20616193.4  20794568.4  -297568.2    -892704.6   

         HUS        HUS1        HUS2    HCO2_1           HCO2_2  \
0  379182.22  368839.049  389847.167  -160.888  -22448.67848156   
1       8.03        8.03        8.03         0                0   
2          0           0           0         0                0   

                REG_DAT